# **Project B - Group 2**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [11]:
import numpy as np
import tensorflow as tf
from PIL import Image
from sklearn.model_selection import train_test_split
from os import listdir
from os.path import isfile, join

## Preprocessing

1) Read the image files

2) Resize them to the specified number of pixel

3) Convert them to grayscale

4) Normalize the values

5) Assign labels to each image


In [61]:
pixel = 32

path1 = './drive/MyDrive/Chondrocytes/'
path2 = './drive/MyDrive/Stemcells/'


#Load Chondrocyte images
i = 0
inp = np.zeros([211,pixel,pixel])
label = np.zeros([211,1])
for f in listdir(path1):
  im = Image.open(join(path1,f))
  im = im.resize((pixel,pixel)).convert('L')
  inp[i,:,:] = np.array(im)/255.0
  label[i] = 1
  i = i+1

print(inp.shape)
print(np.array(im).shape)
print(inp[i,:,:].shape)

(211, 32, 32)
(32, 32)
(32, 32)


In [62]:
#Load Stemcells images
for f in listdir(path2):
  im = Image.open(join(path2,f))
  im = im.resize((pixel,pixel)).convert('L')
  inp[i,:,:] = np.array(im)/255.0
  i = i + 1

inp = inp.reshape((211,pixel,pixel,1))

In [63]:
# Test set
testset = np.zeros([10,pixel,pixel])
path3 =  './drive/MyDrive/Test-both'
j = 0
for f in listdir(path3):
  im = Image.open(join(path3,f))
  im = im.resize((pixel,pixel)).convert('L')
  testset[j] = np.array(im)/255.0
  j = j+1

testset = testset.reshape((10,pixel,pixel,1))

# Data Augmentation
 
 1) Horizontal Flip
 
 2) Vertical Flip

In [64]:
#Data augmentation

aug1 = tf.keras.Sequential(tf.keras.layers.experimental.preprocessing.RandomFlip(mode="horizontal"))
aug2 = tf.keras.Sequential(tf.keras.layers.experimental.preprocessing.RandomFlip(mode="vertical"))

inp2 = np.concatenate([inp,aug1(inp),aug2(inp)],axis = 0)
label2 = np.concatenate([label,label,label],axis = 0)

In [65]:
# Obtaining training and testing data

x1_train,x1_test,y1_train,y1_test = train_test_split(inp,label,test_size=0.2,shuffle =True,random_state=1)
x2_train,x2_test,y2_train,y2_test = train_test_split(inp2,label2,test_size=0.2,shuffle =True,random_state=1)

# Training of CNN

In [66]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Conv2D,Input,MaxPooling2D,Flatten,AveragePooling2D

from tensorflow.keras.callbacks import ModelCheckpoint

In [20]:
#Initial proposal

#LeNet5 Architecture - without augmentation

#Input 32*32 pixels

model1_lenet = Sequential()
model1_lenet.add(Conv2D(6,kernel_size=(5,5),strides=(1,1),activation='relu'))
model1_lenet.add(AveragePooling2D(pool_size=(2,2),strides=(2,2)))
model1_lenet.add(Conv2D(16,kernel_size=(5,5),strides=(1,1),activation='relu'))
model1_lenet.add(AveragePooling2D(pool_size=(2,2),strides=(2,2)))
model1_lenet.add(Flatten())
model1_lenet.add(Dense(100,activation='relu'))
model1_lenet.add(Dense(1,activation='sigmoid'))

model1_lenet.compile(optimizer='adam',loss='binary_crossentropy',metrics='accuracy')

model1_lenet.fit(x1_train,y1_train, validation_data=(x1_test,y1_test),batch_size=5,epochs=1178)

Epoch 1/1214
34/34 [==============================] - 1s 15ms/step - loss: 0.6671 - accuracy: 0.6853 - val_loss: 0.7432 - val_accuracy: 0.5581
Epoch 2/1214
34/34 [==============================] - 0s 9ms/step - loss: 0.6130 - accuracy: 0.7505 - val_loss: 0.7374 - val_accuracy: 0.5581
Epoch 3/1214
34/34 [==============================] - 0s 10ms/step - loss: 0.6336 - accuracy: 0.7177 - val_loss: 0.7809 - val_accuracy: 0.5581
Epoch 4/1214
34/34 [==============================] - 0s 10ms/step - loss: 0.6067 - accuracy: 0.7310 - val_loss: 0.7126 - val_accuracy: 0.5581
Epoch 5/1214
34/34 [==============================] - 0s 9ms/step - loss: 0.6074 - accuracy: 0.7249 - val_loss: 0.7282 - val_accuracy: 0.5581
Epoch 6/1214
34/34 [==============================] - 0s 9ms/step - loss: 0.6073 - accuracy: 0.7143 - val_loss: 0.7272 - val_accuracy: 0.5581
Epoch 7/1214
34/34 [==============================] - 0s 10ms/step - loss: 0.5852 - accuracy: 0.7323 - val_loss: 0.7821 - val_accuracy: 0.5581
Ep

In [21]:
model1_lenet.evaluate(x1_train,y1_train)
model1_lenet.evaluate(x1_test,y1_test)

2/2 [==============================] - 0s 9ms/step - loss: 0.3029 - accuracy: 0.9070


[0.30292245745658875, 0.9069767594337463]

In [49]:
yp1 = model1_lenet.predict(testset)
print('Predicted value')
print((yp1 > 0.5) * 1.0)

Predicted value
[[1.]
 [1.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [1.]]


In [17]:
#LeNet5 Architecture - with augmentation

#Input 32*32 pixels

model2_lenet = Sequential()
model2_lenet.add(Conv2D(6,kernel_size=(5,5),strides=(1,1),activation='relu'))
model2_lenet.add(AveragePooling2D(pool_size=(2,2),strides=(2,2)))
model2_lenet.add(Conv2D(16,kernel_size=(5,5),strides=(1,1),activation='relu'))
model2_lenet.add(AveragePooling2D(pool_size=(2,2),strides=(2,2)))
model2_lenet.add(Flatten())
model2_lenet.add(Dense(100,activation='relu'))
model2_lenet.add(Dense(1,activation='sigmoid'))

model2_lenet.compile(optimizer='adam',loss='binary_crossentropy',metrics='accuracy')

model2_lenet.fit(x2_train,y2_train, validation_data=(x2_test,y2_test),batch_size=5,epochs=1500)

Epoch 1/1500
102/102 [==============================] - 2s 10ms/step - loss: 0.6850 - accuracy: 0.5778 - val_loss: 0.6109 - val_accuracy: 0.7323
Epoch 2/1500
102/102 [==============================] - 1s 8ms/step - loss: 0.6595 - accuracy: 0.6723 - val_loss: 0.6229 - val_accuracy: 0.7323
Epoch 3/1500
102/102 [==============================] - 1s 8ms/step - loss: 0.6375 - accuracy: 0.6777 - val_loss: 0.5946 - val_accuracy: 0.7323
Epoch 4/1500
102/102 [==============================] - 1s 7ms/step - loss: 0.6039 - accuracy: 0.7192 - val_loss: 0.5763 - val_accuracy: 0.7323
Epoch 5/1500
102/102 [==============================] - 1s 8ms/step - loss: 0.5819 - accuracy: 0.6760 - val_loss: 0.4319 - val_accuracy: 0.7323
Epoch 6/1500
102/102 [==============================] - 1s 8ms/step - loss: 0.4280 - accuracy: 0.6741 - val_loss: 0.3418 - val_accuracy: 0.7087
Epoch 7/1500
102/102 [==============================] - 1s 8ms/step - loss: 0.3849 - accuracy: 0.6330 - val_loss: 0.3340 - val_accuracy

In [22]:
model2_lenet.evaluate(x2_train,y2_train)
model2_lenet.evaluate(x2_test,y2_test)

4/4 [==============================] - 0s 9ms/step - loss: 0.5089 - accuracy: 0.9291


[0.5088579058647156, 0.9291338324546814]

In [48]:
yp2 = model2_lenet.predict(testset)
print('Predicted value')
print((yp2 > 0.5) * 1.0)

Predicted value
[[1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [0.]
 [1.]
 [1.]
 [0.]
 [1.]]


In [35]:
#Optimized model without augmentation

model1 = Sequential()
model1.add(Conv2D(16,(7,7),activation='relu'))
model1.add(MaxPooling2D((2,2),strides=2))
model1.add(Conv2D(192,(5,5),activation='relu'))
model1.add(Flatten())
model1.add(Dense(200,activation= 'relu'))
model1.add(Dense(1,activation='sigmoid'))

model1.compile(optimizer='adam',loss='binary_crossentropy',metrics='accuracy')

model1.fit(x1_train,y1_train,validation_data=(x1_test,y1_test),epochs=1200)

Epoch 1/1200
6/6 [==============================] - 1s 111ms/step - loss: 0.6683 - accuracy: 0.7067 - val_loss: 0.8120 - val_accuracy: 0.5581
Epoch 2/1200
6/6 [==============================] - 0s 77ms/step - loss: 0.6219 - accuracy: 0.7464 - val_loss: 0.7223 - val_accuracy: 0.5581
Epoch 3/1200
6/6 [==============================] - 0s 82ms/step - loss: 0.6419 - accuracy: 0.7273 - val_loss: 0.6992 - val_accuracy: 0.5581
Epoch 4/1200
6/6 [==============================] - 0s 78ms/step - loss: 0.6472 - accuracy: 0.7113 - val_loss: 0.7530 - val_accuracy: 0.5581
Epoch 5/1200
6/6 [==============================] - 0s 84ms/step - loss: 0.6251 - accuracy: 0.7223 - val_loss: 0.7943 - val_accuracy: 0.5581
Epoch 6/1200
6/6 [==============================] - 0s 78ms/step - loss: 0.6311 - accuracy: 0.7138 - val_loss: 0.7065 - val_accuracy: 0.5581
Epoch 7/1200
6/6 [==============================] - 0s 81ms/step - loss: 0.6055 - accuracy: 0.7328 - val_loss: 0.7920 - val_accuracy: 0.5581
Epoch 8/1200

In [36]:
model1.evaluate(x1_train,y1_train)
model1.evaluate(x1_test,y1_test)

2/2 [==============================] - 0s 14ms/step - loss: 0.7353 - accuracy: 0.9070


[0.7352913618087769, 0.9069767594337463]

In [47]:
yp3 = model1_lenet.predict(testset)
print('Predicted value')
print((yp3 > 0.5) * 1.0)

Predicted value
[[1.]
 [1.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [1.]]


In [71]:
#Optimized model with augmentation

model2 = Sequential()
model2.add(Conv2D(16,(7,7),activation='relu'))
model2.add(MaxPooling2D((2,2),strides=2))
model2.add(Conv2D(192,(5,5),activation='relu'))
model2.add(Flatten())
model2.add(Dense(200,activation= 'relu'))
model2.add(Dense(1,activation='sigmoid'))

model2.compile(optimizer='adam',loss='binary_crossentropy',metrics='accuracy')
callback = ModelCheckpoint('./MyDrive',monitor="val_accuracy", verbose=0,save_best_only=True,save_weights_only=True)
model2.fit(x2_train,y2_train,validation_data=(x2_test,y2_test),epochs=1500,callbacks=callback)

Epoch 1/1500
16/16 [==============================] - 2s 109ms/step - loss: 0.6874 - accuracy: 0.6066 - val_loss: 0.6242 - val_accuracy: 0.7323
Epoch 2/1500
16/16 [==============================] - 1s 83ms/step - loss: 0.6370 - accuracy: 0.7044 - val_loss: 0.6219 - val_accuracy: 0.7323
Epoch 3/1500
16/16 [==============================] - 1s 84ms/step - loss: 0.6352 - accuracy: 0.6781 - val_loss: 0.5803 - val_accuracy: 0.7323
Epoch 4/1500
16/16 [==============================] - 1s 83ms/step - loss: 0.5847 - accuracy: 0.7165 - val_loss: 0.5442 - val_accuracy: 0.7323
Epoch 5/1500
16/16 [==============================] - 1s 93ms/step - loss: 0.5127 - accuracy: 0.7006 - val_loss: 0.3850 - val_accuracy: 0.7323
Epoch 6/1500
16/16 [==============================] - 1s 79ms/step - loss: 0.4135 - accuracy: 0.7056 - val_loss: 0.3380 - val_accuracy: 0.7953
Epoch 7/1500
16/16 [==============================] - 1s 84ms/step - loss: 0.3740 - accuracy: 0.7791 - val_loss: 0.3480 - val_accuracy: 0.748

In [72]:
model2.load_weights('./MyDrive')
model2.evaluate(x2_train,y2_train)
model2.evaluate(x2_test,y2_test)

4/4 [==============================] - 0s 22ms/step - loss: 0.1537 - accuracy: 0.9685


[0.15367726981639862, 0.9685039520263672]

In [73]:

yp4 = model2_lenet.predict(testset)

print('Predicted value')
print((yp4 > 0.5) * 1.0)

Predicted value
[[1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [0.]
 [1.]
 [1.]
 [0.]
 [1.]]
